# CityBikes

Send a request to CityBikes for the city of your choice.

In [31]:
# imports

import requests
import json # in vscode, this import is needed to parse the JSON from the api call
import pandas as pd
import numpy as np

# from IPython.display import JSON # for nice JSON (or dict) display in JupyterLab (not currently supported in Jupyter Notebook)

Contact the server via an API call

In [32]:
# set city for investigation - Toronto AKA Bike Share Toronto
city_name = "Toronto, ON"

# get network response for all bike share networks in the world first
response = requests.request('GET', url="https://api.citybik.es/v2/networks")
data_networks = response.json()

# normalize it into a dataframe
df_networks_raw = pd.json_normalize(data_networks['networks'])
df_networks_raw

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,ebikes,license.name,license.url
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667,NaN,NaN,NaN,NaN,NaN
1,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,Siena,IT,43.318600,11.330600,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
2,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,Maroussi,GR,38.056872,23.808330,NaN,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,Cyclopolis,Nafplio,GR,37.563940,22.809340,NaN,NaN,NaN,NaN,NaN
4,[Comunicare S.r.l.],/v2/networks/bicincitta-parco-dei-colli-di-ber...,bicincitta-parco-dei-colli-di-bergamo,Bicincittà,Parco dei Colli di Bergamo,IT,45.722956,9.649230,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,[PickeBike],/v2/networks/pickebike-aubonne,pickebike-aubonne,PickeBike Aubonne,Aubonne,CH,46.494900,6.396300,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/pi...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...
786,"[Mobilitätsakademie AG, Touring Club Switzerla...",/v2/networks/carvelo,carvelo,carvelo eCargo-Bike Sharing,Switzerland,CH,46.982900,8.233100,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/ca...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...
787,[PickeBike],/v2/networks/pickebike-basel,pickebike-basel,PickeBike Basel,Basel,CH,47.557900,7.590200,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/pi...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...
788,[PickeBike],/v2/networks/pickebike-fribourg,pickebike-fribourg,PickeBike Fribourg,Fribourg,CH,46.803900,7.150500,https://github.com/SFOE/sharedmobility/blob/ma...,https://gbfs.prod.sharedmobility.ch/v2/gbfs/pi...,NaN,Open Use,https://opendata.swiss/en/dataset/standorte-un...


790 Cities in lots of countries, I want to find Toronto, Canada

In [33]:
# Country is a two letter symbol so search Canada with CA
df_networks_raw[df_networks_raw['location.country'] == 'CA']

# Toronto is found under 'Toronto, ON' not 'Toronto'

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,ebikes,license.name,license.url
70,"[Motivate International, Inc., PBSC Urban Solu...",/v2/networks/bixi-montreal,bixi-montreal,Bixi,"Montréal, QC",CA,45.508693,-73.553928,NaN,https://gbfs.velobixi.com/gbfs/gbfs.json,True,NaN,NaN
72,"[Motivate International, Inc., PBSC Urban Solu...",/v2/networks/bixi-toronto,bixi-toronto,Bike Share Toronto,"Toronto, ON",CA,43.653226,-79.383184,NaN,https://tor.publicbikesystem.net/ube/gbfs/v1/,NaN,NaN,NaN
73,[Social Bicycles Inc.],/v2/networks/sobi-hamilton,sobi-hamilton,SoBi,"Hamilton, ON",CA,43.256436,-79.869297,NaN,https://hamilton.socialbicycles.com/opendata/g...,NaN,NaN,NaN
199,"[Vanncouver Bike Share Inc., CycleHop LLC, Cit...",/v2/networks/mobibikes,mobibikes,Mobi,Vancouver,CA,49.282700,-123.120700,NaN,https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs....,NaN,NaN,NaN
504,[PBSC Urban Solutions],/v2/networks/avelo-quebec,avelo-quebec,àVélo,Québec,CA,46.813050,-71.216800,NaN,https://quebec.publicbikesystem.net/customer/g...,NaN,NaN,NaN
505,[PBSC Urban Solutions],/v2/networks/acces-velo-saguenay,acces-velo-saguenay,Accès Vélo,Saguenay,CA,48.433333,-71.083333,NaN,https://saguenay.publicbikesystem.net/customer...,NaN,NaN,NaN


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes).

In [34]:
# we have the href for toronto /v2/networks/bixi-toronto and according to api documentation we make the call by adding fields ?field=list,of,fields

href_toronto = "/v2/networks/bixi-toronto"

response = requests.request('GET', url="https://api.citybik.es/v2/networks/bixi-toronto?fields=stations")
data_stations = response.json()


# NOTE: Current call was made at 5:30pm on Tuesday afternoon in late July
# Excellent warm clear weather, after work dinner/drinks rush, excellent conditions for cycling downtown

Put your parsed results into a DataFrame.

In [35]:
df_toronto_stations = pd.json_normalize(data_stations['network']['stations'])
df_toronto_stations.head(10)

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.altitude,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.payment,extra.payment-terminal,extra.renting,extra.returning,extra.slots,extra.uid,extra.post_code
0,7,12,fb337bbed72e2be090071e199899b2be,43.665269,-79.319796,Queen St E / Woodward Ave,2024-07-30T21:37:13.785000Z,Queen St E / Woodward Ave,0.0,0,True,1722375262,12,"[key, transitcard, creditcard, phone]",True,1,1,19,7303,NaN
1,14,1,4ff88d5880e71aa40d34cfe5d09b0ca7,43.671420,-79.445947,Primrose Ave / Davenport Rd,2024-07-30T21:37:13.797000Z,Primrose Ave / Davenport Rd,0.0,0,True,1722375345,1,"[key, transitcard, creditcard, phone]",True,1,1,15,7301,NaN
2,15,6,a09c67c0b419654d907c9134b108e328,43.666224,-79.317693,Queen St E / Rhodes Ave,2024-07-30T21:37:13.825000Z,Queen St. E / Rhodes Ave.,0.0,0,True,1722375333,6,"[key, transitcard, creditcard, phone]",True,1,1,23,7309,NaN
3,34,2,d6a9daee68070a8b106cfb598d81308c,43.653236,-79.376716,Bond St / Queen St E,2024-07-30T21:37:13.946000Z,Bond St / Queen St E,0.0,1,True,1722375304,1,"[key, transitcard, creditcard, phone]",True,1,1,37,7045,NaN
4,21,12,8f8af40d9388c8a3962559e8681d3db7,43.663722,-79.380288,Church St / Alexander St,2024-07-30T21:37:13.968000Z,Church St / Alexander St,0.0,2,True,1722375423,10,"[key, transitcard, creditcard, phone]",True,1,1,35,7044,NaN
5,13,7,5a441560e262305d1934fb3ada5a9517,43.657763,-79.389165,University Ave / Gerrard St W (East Side),2024-07-30T21:37:13.981000Z,University Ave / Gerrard St W,0.0,0,True,1722375314,7,"[key, transitcard, creditcard, phone]",True,1,1,25,7047,NaN
6,4,17,18ea490ab3ca4d10e2927308416edbef,43.656729,-79.382736,Edward St / Yonge St,2024-07-30T21:37:13.998000Z,Edward St / Yonge St,NaN,0,True,1722375337,17,"[key, transitcard, creditcard, phone]",True,1,1,22,7041,NaN
7,14,2,b7850948607474362d3728f19a77fdff,43.664467,-79.414783,Euclid Ave / Bloor St W,2024-07-30T21:37:14.021000Z,Euclid Ave / Bloor St W,0.0,1,True,1722375311,1,"[key, transitcard, creditcard, phone]",True,1,1,19,7040,NaN
8,22,0,7ffd06e0207f80fe1ff6b5ae0bb0544c,43.675492,-79.388858,Yonge St / Aylmer Ave,2024-07-30T21:37:14.028000Z,Yonge St / Aylmer Ave,0.0,0,True,1722375251,0,"[key, transitcard, creditcard, phone]",True,1,1,23,7128,NaN
9,10,0,17fadbdc5848036c8485099bbf517294,43.674991,-79.396273,Davenport Rd / Avenue Rd,2024-07-30T21:37:14.036000Z,Davenport Rd / Avenue Rd,0.0,0,True,1722375420,0,"[key, transitcard, creditcard, phone]",True,1,1,11,7129,NaN


In [36]:
# extract the columns of interest into a new df to strip out the irrelevant/extra columns

df_toronto = df_toronto_stations[['latitude', 'longitude', 'free_bikes', 'empty_slots', 'name']]
df_toronto.head(10)

,latitude,longitude,free_bikes,empty_slots,name
0,43.665269,-79.319796,12,7,Queen St E / Woodward Ave
1,43.671420,-79.445947,1,14,Primrose Ave / Davenport Rd
2,43.666224,-79.317693,6,15,Queen St E / Rhodes Ave
3,43.653236,-79.376716,2,34,Bond St / Queen St E
4,43.663722,-79.380288,12,21,Church St / Alexander St
5,43.657763,-79.389165,7,13,University Ave / Gerrard St W (East Side)
6,43.656729,-79.382736,17,4,Edward St / Yonge St
7,43.664467,-79.414783,2,14,Euclid Ave / Bloor St W
8,43.675492,-79.388858,0,22,Yonge St / Aylmer Ave
9,43.674991,-79.396273,0,10,Davenport Rd / Avenue Rd
